In [2]:
import sys
import os

from dotenv import load_dotenv

sys.path.append(os.path.join(os.path.dirname(os.getcwd()), "src"))

from mav.MAS.agents import (
    Runner,
    InMemorySession,
    BaseSession,
    RunResult,
    Agent
)

from mav.Tasks.base_environment import TaskEnvironment


load_dotenv()

True

In [3]:
def get_weather(location: str) -> str:
    """
    Tool to get the weather for a given location.
    Args:
        location (str): The city to get the weather for.
    
    """
    return f"The weather in {location} is sunny with a high of 25°C."

In [4]:
agent = Agent(
    name="Weather Agent",
    model="gpt-4.1",
    instructions="You are an intelligent assistant.",
    model_settings={"temperature": 0.0},
    tools = [get_weather]
)

agent.tools

[{'type': 'function',
  'function': {'name': 'get_weather',
   'description': 'Tool to get the weather for a given location.',
   'parameters': {'type': 'object',
    'properties': {'location': {'type': 'string',
      'description': 'The city to get the weather for.'}},
    'required': ['location']}}}]

In [5]:
memory = InMemorySession(session_id="weather_session")
await memory.add_items([{"role": "user", "content": "Hi, how are you today?"}, {"role": "assistant", "content": "I'm doing well, thank you! How can I assist you today?"}])
await memory.get_items()

[{'role': 'user', 'content': 'Hi, how are you today?'},
 {'role': 'assistant',
  'content': "I'm doing well, thank you! How can I assist you today?"}]

In [6]:
result = await Runner.run(
    agent = agent,
    input = "what is the weather in New York?",
    context=TaskEnvironment(),
)

In [11]:
result.usage

{'completion_tokens': 32,
 'prompt_tokens': 178,
 'total_tokens': 210,
 'completion_tokens_details': {'accepted_prediction_tokens': 0,
  'audio_tokens': 0,
  'reasoning_tokens': 0,
  'rejected_prediction_tokens': 0},
 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}

In [6]:
await memory.get_items()

[{'role': 'user', 'content': 'Hi, how are you today?'},
 {'role': 'assistant',
  'content': "I'm doing well, thank you! How can I assist you today?"}]

In [7]:
result = await Runner.run(
    agent = agent,
    input = "Would you say this temperature is comfortable for a walk in the park of New York?",
    context=TaskEnvironment(),
    session=memory
)

In [8]:
result.final_output

'Yes, a temperature of 25°C (77°F) with sunny weather in New York is generally considered very comfortable for a walk in the park. It’s warm but not too hot, making it ideal for outdoor activities. Would you like any tips for your walk or information about parks in New York?'

In [9]:
await memory.get_items()

[{'role': 'user', 'content': 'Hi, how are you today?'},
 {'role': 'assistant',
  'content': "I'm doing well, thank you! How can I assist you today?"},
 {'role': 'user',
  'content': 'Would you say this temperature is comfortable for a walk in the park of New York?'},
 {'content': None,
  'role': 'assistant',
  'tool_calls': [{'function': {'arguments': '{"location":"New York"}',
     'name': 'get_weather'},
    'id': 'call_MFMai9A3eDqNOnusxDn7YIwG',
    'type': 'function'}],
  'function_call': None,
  'annotations': []},
 {'tool_call_id': 'call_MFMai9A3eDqNOnusxDn7YIwG',
  'role': 'tool',
  'name': 'get_weather',
  'content': 'The weather in New York is sunny with a high of 25°C.'},
 {'content': 'Yes, a temperature of 25°C (77°F) with sunny weather in New York is generally considered very comfortable for a walk in the park. It’s warm but not too hot, making it ideal for outdoor activities. Would you like any tips for your walk or information about parks in New York?',
  'role': 'assista